# Flagging


Radio interferometry data analysis applications and algorithms may be assembled from CNGI and ngCASA building blocks. A user may choose to implement their own analysis scripts, use a pre-packaged task similar to those in current CASA or embed ngCASA and CNGI methods in a production pipeline DAG.

Note : The following examples represent preliminary design ideas that illustrate how ngCASA science applications may be assembled. The API and usage example scripts will change as the details are refined. The current focus is to evaluate whether the CNGI and ngCASA functional design and infrastructure adequately addresses algorithmic needs and how it may be best leveraged for scaleable high performance computing. Questions raised via this initial exercise will guide the design of [future prototypes](https://cngi-prototype.readthedocs.io/en/latest/development.html#List-of-Future-Prototypes), continued evaluation the chosen infrastructure, and the final function hierarchy and API definition. 

Please note that these examples are currently notional, and not yet runnable in the colab!

The examples below illustrate some flagging operations. 

Flags are stored as Boolean arrays in the zarr and xarray datasets. Flag versions are maintained by giving these arrays names. Each flagging method can write to a specified named flag array. Applications that use flags will read from the specified flag array name and set corresponding data array values to NaN (using cngi.vis.applyflags()) before proceeding.
See : https://cngi-prototype.readthedocs.io/en/latest/visibilities.html#Flagging

Interactive flag visualization may be done at the application layer by inserting a plotting/visualization step in between any of the calls to ngcasa.flagging methods. Flag versions may be managed (merged, copied, deleted) using the ngcasa.flagging.manage_flags() method. For the inevitable experimentation required to tune autoflag parameters, a named flag array may be created, used, visualized, and then discarded. 



## Manual and meta-data based flags
Online flags typically consist of many (1000s) of data selection queries that mark regions to be flagged. 

Shadow and elevation flagging is also typically done along with manual flag selections.


In [ ]:
#Construct a selected vis dataset
vis_dataset = cngi.dio.read_vis(visname, selpars)

#Define manual flags using the 'isel' syntax that references keywords in the zarr/xds
# https://cngi-prototype.readthedocs.io/en/latest/visibilities.html#Selection-and-Splitting

list_sels = [{'time':[76,77,78], 'chan':[6,7,8,12]},
             {'time':[112,113], 'chan':[6,7,56]}]

# Set the FLAG to 1 for all points in the union of all selections
ngcasa.flagging.manual_flag(vis_dataset,list_sels, flag_name='FLAG')

# Calculate shadow and elevation flags
ngcasa.flagging.elevation(vis_dataset)
ngcasa.flagging.shadow(vis_dataset)

Note that manual flags are restricted to data selections only. The casa6 concept of 'flag commands' that represent other types of flagging will not be followed here. 

## Autoflag with extension and pre-existing flags
This example demonstrates the use-case of extending flags generated only by
the autoflag algorithm, but not all manually-set pre-existing flags. 

This is a use-case currently not possible in the casa6 flagger framework. 

In [ ]:
#Construct a selected vis dataset
vis_dataset = cngi.dio.read_vis(visname, selpars)

# Do some manual flagging and save the flags to the FLAG column. 
ngcasa.flagging.manual_flag(vis_dataset, list_sels, out_flag_name='FLAG')

# Run the rflag algorithm using FLAG as the pre-existing flags,
# Write only the new flags into a new FLAG_AUTO array.
ngcasa.flagging.auto_rflag(vis_dataset_avg,algopars,
                           in_flag_name='FLAG',
                           out_flag_name='FLAG_AUTO')

# Extend only the new autoflags, but not all pre-existing flags
ngcasa.flagging.extend(vis_dataset,extendpars, 
                    in_flag_name='FLAG_AUTO',
                    out_flag_name='FLAG_AUTO')

# Now, merge the flags using a logical OR, and save it into the default 'FLAG' array
ngcasa.flagging.manage_flags(vis_dataset,
                             in_flags=['FLAG','FLAG_AUTO'], 
                             out_flag_name='FLAG', 
                             op='or')

## < Visualize the flags by plotting the data with a chosen flagversion > 
## < If unsatified, discard flagversion using a cngi.dio.xxxx step and repeat the above >

## Save vis_dataset to zarr.
cngi.dio.write_zarr(vis_dataset)

## Autoflag with pre-averaging
This example demonstrates averaging for autoflagging, expanding the flags back to the original dataset. Two manual flag calls are also included, one on the original data and one after averaging.

Flag expansion is done as a regrid operation. 

In [ ]:
# Construct a selected vis dataset
vis_dataset = cngi.dio.read_vis(visname, selpars)

# (1) Do some manual flagging and save the flags to the FLAG column. 
ngcasa.flagging.manual_flag(vis_dataset, list_sels_original, out_flag_name='FLAG')

# Now, average the data in both time and frequency prior to running autoflag.
vis_dataset_time_avg = cngi.vis.timeaverage(vis_dataset)
vis_dataset_time_freq_avg = cngi.vis.chanaverage(vis_dataset_time_avg)

# (2) Run the tfcrop algorithm using FLAG as the pre-existing flags
# and save the new flags into the same flag array.
ngcasa.flagging.auto_tfcrop(vis_dataset_time_freq_avg,algopars,
                           in_flag_name='FLAG',
                           out_flag_name='FLAG')

# (3) Manual flags, using meta-data corresponding to the averaged-data.
ngcasa.flagging.manual_flag(vis_dataset_time_freq_avg, list_sels_lowres, out_flag_name='FLAG')

# (4) Manual unflag, using meta-data corresponding to the averaged-data
ngcasa.flagging.manual_unflag(vis_dataset_time_freq, list_sels_lowres_unflag, out_flag_name='FLAG')


# Now, expand the flags back to the original resolution. 
#        Note that Step (1) was already on the original data. 
#        The results of Steps (2) and (3) and (4) will get expanded out. 
#
## TBD : Need a demo at the CNGI level of how to expand flags back to the original data
##       Expansion is a regrid where the same value is repeated across the expanded range.
cngi.vis.regrid(in_xds=vis_dataset_time_freq, out_xds=vis_dataset)

# Save to disk
cngi.dio.write_zarr(vis_dataset)

## FlagVersion handling for pipelines
Save and restore flag versions, along with dataset selections/splits.

The use case is based on current operations with casa6, where parallelization is implemented by partitioning the data and running operations that set flags, on each subset.  It is TBD whether this use-case is still relevant with CNGI and ngCASA, but here is an example to showcase how this may be achieved. 

TBD : Need input from pipeline group : Does this represent a relevant use case ? 

In [ ]:
#Construct several selected vis datasets
vis_dataset_target1 = cngi.dio.read_vis(visname, selpars_target1)
vis_dataset_target2 = cngi.dio.read_vis(visname, selpars_target2)

# Do different operations on the two datasets
cngi.ngcasa.autoflag_rflag(vis_dataset_target1)
cngi.ngcasa.manual_flag(vis_dataset_target2,list_sels_to_flag)

# Regrid/expand the flags back to the original dataset
cngi.vis.regrid(in_xds=vis_dataset_target1, out_xds=vis_dataset)
cngi.vis.regrid(in_xds=vis_dataset_target2, out_xds=vis_dataset)

# Save to disk
cngi.dio.write_zarr(vis_dataset)